<a href="https://colab.research.google.com/github/adamhecktman/ML/blob/master/Stopword_Process_Verified_Programs_Outcomes_BERTforSequenceClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch


# Load file and convert pandas series to lists.

In [4]:
myFile = "/content/verified program descirptions and outcomes and handcoded.csv"
df = pd.read_csv(myFile)

texts = df['Descriptions'].tolist()
labels = df['Outcomes'].tolist()

# Remove stopwords from texts list

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [6]:
# Define a function to remove stop words
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Sample list of text
text_list = texts

# Remove stopwords from each text in the list
cleaned_text_list = [remove_stopwords(text) for text in text_list]




# Ensure labels are encoded as integers (which they are) - Skipped this

In [7]:
unique_labels = list(set(labels))
label_map = {label: idx for idx, label in enumerate(unique_labels)}
labels = [label_map[label] for label in labels]

In [8]:
setLabels = set(labels)

#Initalize the BERT tokenizer

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

# Tokenize the Texts


In [10]:
encodings = tokenizer(cleaned_text_list, truncation=True, padding=True, max_length=128, return_tensors='pt', add_special_tokens = True)

# Convert labels to tensor

In [11]:
labels = torch.tensor(labels)

# Create the dataset class


In [12]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Create the dataset using encodings and labels

In [13]:
dataset = TextDataset(encodings, labels)


In [14]:
len(dataset)

32417

# Split test/train data

In [15]:
train_size = 0.8
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [int(train_size * len(dataset)), len(dataset) - int(train_size * len(dataset))])


# Load BertForSequenceClassification

In [16]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(unique_labels))
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#Train the model

In [17]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,4.367900
1000,3.225300
1500,2.963700
2000,2.834300
2500,2.705400
3000,2.664800
3500,2.524900
4000,2.402600
4500,2.319200
5000,2.296500


TrainOutput(global_step=9726, training_loss=2.4026129046217934, metrics={'train_runtime': 1199.1137, 'train_samples_per_second': 64.88, 'train_steps_per_second': 8.111, 'total_flos': 5130814961887488.0, 'train_loss': 2.4026129046217934, 'epoch': 3.0})

In [21]:
results = trainer.evaluate()
print(results)

{'eval_loss': 2.4462273120880127, 'eval_runtime': 24.2263, 'eval_samples_per_second': 267.643, 'eval_steps_per_second': 33.476, 'epoch': 3.0}


In [23]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Get predictions from the model
predictions = trainer.predict(val_dataset)


# Evaluate and print results

In [24]:
# prompt: how do i get an accuracy score from transformers.trainer_utils.PredictionOutput

from transformers.trainer_utils import PredictionOutput
import numpy as np

# Assuming 'predictions' is your PredictionOutput object
predicted_labels = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids

accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy}")

Accuracy: 0.44524984577421345
